# Comparing EU PAS Register and EMA RWD Catalogue data
We will compare the scraped EU PAS Register Data and the scraped EMA RWD Catalogue data to find out changes made to the `centre` and `funding` fields.

<small>**NOTE:** We did use this notebook to figure out which columns we will use for the sponsor matching.</small>

<small>**NOTE:** `centre_name` and `centre_name_of_investigator` were used previously for the EU PAS Register.</small>

<small>**NOTE:** `funding_details` was used for the EMA RWD Catalalogue.</small>

First we import `pandas`:

In [1]:
import pandas as pd

We will now read the scraped data of the EU PAS Register, the EMA RWD Catalogue and the centre matching file and filter the relevant columns.

In [ ]:
eupas = (
    pd.read_csv('./data/scraped_eupas_2024-01-23T22-15-21+00-00.csv')
    .set_index('eu_pas_register_number')
    .filter(like='centre')
    .drop('centre_location', axis='columns')
)
eupas

,centre_name,centre_name_of_investigator,centre_organisation
eu_pas_register_number,,,
49988,Eli Lilly and Company,NaN,NaN
11512,NaN,OPRI Pte Ltd,OPRI Pte Ltd
9142,NaN,OPRI Pte Ltd,OPRI Pte Ltd
7678,NaN,OPRI Pte Ltd,OPRI Pte Ltd
104156,NaN,OPRI Pte Ltd,OPRI Pte Ltd
...,...,...,...
6023,NaN,PAREXEL,PAREXEL International Corporation
13040,NaN,OXON,Oxon Epidemiology Ltd.
35147,NaN,OXON,OXON Epidemiology Ltd.


In [ ]:
na_values = [
    "", "#N/A", "#N/A N/A", "#NA", "-1.#IND", "-1.#QNAN", "-NaN", "-nan", 
    "1.#IND", "1.#QNAN", "<NA>", "NULL", "NaN", "None", "nan", "null"
    # "N/A",
    # "NA",
    # "n/a",
]

matching_data = pd.read_excel(
    '../../data/eupas/centre_manual.xlsx', #
    sheet_name=None, 
    keep_default_na=False,
    na_values=na_values,
    na_filter=True
)

In [ ]:
ema_rwd = (
    pd.read_csv('./data/scraped_ema-rwd_2024-02-21T22-22-05+00-00.csv')
    .set_index('eu_pas_register_number')
    .filter(regex='institution|network|funding_details')
)
ema_rwd

,additional_institutions_encepp,additional_institutions_not_encepp,funding_details,lead_institution_encepp,lead_institution_not_encepp,networks_encepp,networks_not_encepp
eu_pas_register_number,,,,,,,
1587,NaN,NaN,"Ministry of Science & Technology, Vall d'Hebro...",University Hospital Vall d’Hebron (HUVH),NaN,NaN,NaN
3061,NaN,NaN,Boehringer Ingelheim,Brigham and Women's Hospital,NaN,NaN,NaN
8800,NaN,NaN,"Best Care Consulting, MSC-INHPs",Best Care Consulting,NaN,NaN,MSC-INHP
3177,NaN,Multiple centres: 999 centres are involved in ...,Daiichi Sankyo European affiliates and Daiichi...,Institut für Pharmakologie und präventive Medizin,Dr. Bramlage & Dr. Hankowitz Partnerschaft,NaN,NaN
9215,NaN,NaN,National health insurance,Institute of Oncology Slovenia,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2113,NaN,Multiple centres: 9 centres involved in this s...,F. Hoffmann-La Roche Ltd,"Scientific Affairs, Outcome SARL",NaN,NaN,Full list available on request
2572,NaN,NaN,Cancer Research UK,Queen's University Belfast,NaN,NaN,NaN
2221,"Centre for Maternal, Fetal and Infant Research...","University of Groningen (RUG) Netherlands, The...","FP7, Universities",University of Ulster,NaN,European Surveillance of Congenital Anomalies ...,NaN


## Deleted and new Studies
We will now analysed which studies got deleted or added in the time between these two datasets:

In [7]:
shared_index = eupas.index.intersection(ema_rwd.index)
print('Removed from EU PAS:', eupas.index.difference(ema_rwd.index).values)
print('New in EMA RWD:', ema_rwd.index.difference(eupas.index).values)
eupas = eupas.filter(items=shared_index, axis='index')
ema_rwd = ema_rwd.filter(items=shared_index, axis='index')

Removed from EU PAS: [12194 23583 29718 30525 35008 35763 39572 45442 47920 49370 50259]
New in EMA RWD: []


## Match Centre Names
We will also match the centre names with the old matching file.

In [8]:
for field_name in ['centre_name', 'centre_name_of_investigator']:
    
    eupas = pd.merge(
        eupas.reset_index(),
        matching_data[field_name].loc[:, ['manual', 'original']].rename(
            columns={
                'manual': f'$MATCHED_{field_name}',
                'original': field_name
            }
        ),
        how='left',
        on=field_name,
    ).set_index('eu_pas_register_number')

match_combined_field_name = '$MATCHED_combined_centre_name'

eupas[match_combined_field_name] = \
    eupas.filter(like='$MATCHED') \
    .apply(lambda x: ''.join([str(y) for y in x.values if isinstance(y, str)]), axis='columns')

eupas.loc[
    eupas[match_combined_field_name] == '',
    match_combined_field_name
] = pd.NA

eupas = eupas.drop(['$MATCHED_centre_name_of_investigator', '$MATCHED_centre_name'], axis='columns')
column = eupas.pop(match_combined_field_name)
eupas.insert(0, match_combined_field_name, column)

eupas

,$MATCHED_combined_centre_name,centre_name,centre_name_of_investigator,centre_organisation
eu_pas_register_number,,,,
49988,Eli Lilly,Eli Lilly and Company,NaN,NaN
11512,Observational and Pragmatic Research Institute...,NaN,OPRI Pte Ltd,OPRI Pte Ltd
9142,Observational and Pragmatic Research Institute...,NaN,OPRI Pte Ltd,OPRI Pte Ltd
7678,Observational and Pragmatic Research Institute...,NaN,OPRI Pte Ltd,OPRI Pte Ltd
104156,Observational and Pragmatic Research Institute...,NaN,OPRI Pte Ltd,OPRI Pte Ltd
...,...,...,...,...
6023,Parexel,NaN,PAREXEL,PAREXEL International Corporation
13040,OXON Epidemiology,NaN,OXON,Oxon Epidemiology Ltd.
35147,OXON Epidemiology,NaN,OXON,OXON Epidemiology Ltd.


## Field changes
We can now conveniently analyse the changes of the `centre` fields:

In [9]:
merged = pd.merge(eupas, ema_rwd, left_index=True, right_index=True)
merged

,$MATCHED_combined_centre_name,centre_name,centre_name_of_investigator,centre_organisation,additional_institutions_encepp,additional_institutions_not_encepp,funding_details,lead_institution_encepp,lead_institution_not_encepp,networks_encepp,networks_not_encepp
eu_pas_register_number,,,,,,,,,,,
49988,Eli Lilly,Eli Lilly and Company,NaN,NaN,NaN,NaN,Eli Lilly and Company,Eli Lilly and Company,NaN,NaN,NaN
11512,Observational and Pragmatic Research Institute...,NaN,OPRI Pte Ltd,OPRI Pte Ltd,NaN,NaN,Teva,Observational & Pragmatic Research Institute P...,Foundation for the Promotion of Health and Bio...,NaN,NaN
9142,Observational and Pragmatic Research Institute...,NaN,OPRI Pte Ltd,OPRI Pte Ltd,NaN,NaN,Chiesi Ltd,Observational & Pragmatic Research Institute P...,Biogen,NaN,NaN
7678,Observational and Pragmatic Research Institute...,NaN,OPRI Pte Ltd,OPRI Pte Ltd,NaN,NaN,TEVA Ltd,Observational & Pragmatic Research Institute P...,Baxter Healthcare Corporation,NaN,NaN
104156,Observational and Pragmatic Research Institute...,NaN,OPRI Pte Ltd,OPRI Pte Ltd,NaN,NaN,Astra Zeneca,Observational & Pragmatic Research Institute P...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6023,Parexel,NaN,PAREXEL,PAREXEL International Corporation,NaN,NaN,Viatris Specialty LLC,Parexel International,NaN,NaN,NaN
13040,OXON Epidemiology,NaN,OXON,Oxon Epidemiology Ltd.,NaN,NaN,Medical Developments International,OXON Epidemiology,NaN,NIHR Medicines for Children Research Network,NaN
35147,OXON Epidemiology,NaN,OXON,OXON Epidemiology Ltd.,NaN,NaN,"GSK, OXON Epidemiology",OXON Epidemiology,NaN,NaN,NaN


We can also export the data with the following code for easy access:

In [10]:
merged.to_excel('compare_sponsors.xlsx')